In [1]:
import pandas as pd
import openpyxl as xl
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import json
import geojson
import requests

In [2]:
df = pd.read_excel('Georreferenciado.xlsx', sheet_name='CoordAdjust')

In [ ]:
df.head(1)

In [ ]:
df1 = df.drop(columns='indice')
df1.head(1)

In [5]:
df2 = pd.read_excel('RouboRecupVec2.xlsx')

In [6]:
df2.head(1)
df2.shape

(512081, 21)

In [12]:
filtro = (pd.notna(df2['Geocode'])&
          (df2['Tipo Fato']=='Consumado') &
          ((df2['Município']=='Porto Alegre')|
          (df2['Município']=='Canoas')|
          (df2['Município']=='Viamão')|
          (df2['Município']=='Cachoeirinha')|
          (df2['Município']=='Gravataí')|
          (df2['Município']=='São Leopoldo')|
          (df2['Município']=='Novo Hamburgo')|
          (df2['Município']=='Alvorada')|
          (df2['Município']=='Sapucaia do Sul')|
          (df2['Município']=='Guaíba')|
          (df2['Município']=='Eldorado do Sul')|
          (df2['Município']=='Esteio')))

In [13]:
df3 = df2.loc[filtro]
df3.shape

(71332, 21)

In [14]:
filtro2=(pd.notna(df2['Geocode'])&
          (df2['Tipo Fato']=='Consumado') &
          (df2['Município']=='Porto Alegre') & (df2.Ano==2018))
dfT = df2.loc[filtro2]
dfT.shape

(13612, 21)

In [15]:
def coordToArray(geopdcoord):
    i = geopdcoord.split(",") # splita resultando num array com as 2 coord
    ponto = Point(float(i[1]),float(i[0])) #ordem invertida - base original lat long, programas funcionam long lat
    return ponto

In [16]:
coordenadas = []
for coord in df3['Geocode']:
    coordenadas.append(coordToArray(str(coord)))
len(coordenadas)

71332

In [ ]:
df3['Geocode Novo'] = coordenadas
df3['Geocode Novo'].shape

In [ ]:
df3.head(3)

In [19]:
cidades = {'municípios':['Porto Alegre',
                         'Canoas',
                         'Viamão',
                         'Cachoeirinha',
                         'Gravataí',
                         'São Leopoldo',
                         'Novo Hamburgo',
                         'Alvorada',
                         'Sapucaia do Sul',
                         'Guaíba',
                         'Eldorado do Sul',
                         'Esteio'],
           'código':[4314902,
                     4304606,
                     4323002,
                     4303103,
                     4309209,
                     4318705,
                     4313409,
                     4300604,
                     4320008,
                     4309308,
                     4306767,
                     4307708]}     
       
dfCodCidades =pd.DataFrame(cidades)

In [20]:
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

arrayShapes = []
for cod in cidades['código']:
    shapes = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/"+str(cod)+"/?formato=application/vnd.geo+json&resolucao=5",
                              headers=headers)
    #atribui string json do shape para variável
    arrayShapes.append(Polygon(list(geojson.utils.coords(shapes.json()))))
    
dfCodCidades['ShapesIBGE'] = arrayShapes

In [21]:
ajuste = []
for x, y in zip(df3['Município'], df3['Geocode Novo']):
    #procura no df na a coluna onde o o municipio é igual x e retorna o valor da terceira coluna que tem os Shapes do IBGE
    poly = dfCodCidades.loc[dfCodCidades['municípios']==x].iloc[0][2]
    #verifica se o ponto y  está n dentro do shape da cidade e adicona o resulatado em um array
    ajuste.append(poly.contains(y))

In [22]:
df3 = df3.drop(columns='Ajustar Coordenadas') #deleta antiga coluna ajustar colunas
df3['Ajustar Coordenadas'] = ajuste #adicona nova coluna com a classificação

In [28]:
filtro23=((df1['Tipo Fato']=='Consumado') &
          (df1['Município']=='Porto Alegre')& (df1.Ano==2018))
dfT1 = df1.loc[filtro23]
dfT1.shape

(58, 21)

In [31]:
#base com dados procergs + minha compilação
df4 = df1
dfJoinned = df4.append(df3, ignore_index = True)

In [33]:
with pd.ExcelWriter('BaseGeorrefProcergsLucas.xlsx') as writer: 
    dfJoinned.to_excel(writer, sheet_name='Base')

In [36]:
#separando roubo e recuperacao, e apenas coordenadas corretas
filtro2 = ((dfJoinned['Fato']!="RECUPERACAO DE VEICULO") & (dfJoinned['Ajustar Coordenadas']==True))
dfRoubo = dfJoinned.loc[filtro2]
filtro3 = ((dfJoinned['Fato']=="RECUPERACAO DE VEICULO") & (dfJoinned['Ajustar Coordenadas']==True))
dfRecup = dfJoinned.loc[filtro3]

In [37]:
print('Base de Roubo a veículo: {} /n base Recuperação {}'.format(dfRoubo.shape,dfRecup.shape))

Base de Roubo a veículo: (105218, 21) /n base Recuperação (106943, 21)


In [ ]:
dfRecup2 = dfRecup.drop(columns=['Ano Registro','Número Inteiro Ocorrência','Bairro Fato','Geocode','Tipo Fato','Ajustar Coordenadas','STATUS'])
#mudando o nome das colunas para poder fazer o merge sem perdas depois
dfRecup2.columns =['ÓrgReg',
                  'ano recup',
                  'Hora recup',
                  'Placa',
                  'marca',
                  'Tipo Veíc',
                  'Ano Fabr',
                  'Ano Model',
                  'Mun recup',
                  'Dt recup',
                  'Rua recup',
                  'Nro recup',
                  'fato',
                  'Geocode recup']
dfRecup2.head(2)

In [40]:
#COMBINANDO LINHAS DE ROUBO COM LINHAS DE RECUPERAÇAO
dfRoubo.set_index('Placa')
dfRecup2.set_index('Placa')
dfRelacao = pd.merge(dfRoubo,dfRecup2)

In [41]:
dfRelacao.columns

Index(['Órgão Registro', 'Ano Registro', 'Número Inteiro Ocorrência', 'Ano',
       'Hora Minuto', 'Bairro Fato', 'Geocode', 'Placa', 'Marca',
       'Tipo de Veículo', 'Ano Fabricação', 'Ano Modelo', 'Município',
       'Data Fato', 'Nome Rua', 'Nro', 'Tipo Fato', 'Fato', 'Geocode Novo',
       'STATUS', 'Ajustar Coordenadas', 'ÓrgReg', 'ano recup', 'Hora recup',
       'marca', 'Tipo Veíc', 'Ano Fabr', 'Ano Model', 'Mun recup', 'Dt recup',
       'Rua recup', 'Nro recup', 'fato', 'Geocode recup'],
      dtype='object')

In [ ]:
dfRel = dfRelacao.drop(columns=['Ano Registro',
                                'Número Inteiro Ocorrência',
                                'Bairro Fato',
                                'Geocode',
                                'Tipo Fato',
                                'Ajustar Coordenadas',
                                'STATUS',
                                'Bairro Fato',
                                'Ano Fabr', 
                                'Ano Model',
                                'Tipo Fato',
                                'Rua recup', 
                                'Nro recup',
                                'Tipo de Veículo',
                                'Ano Fabricação', 
                                'Ano Modelo',
                                'Tipo Veíc', 
                                'Ano Fabr', 
                                'Ano Model',
                                'ÓrgReg',
                                'marca'])
dfRel.head(2)

In [51]:
def coordToPoint(geopdcoord):
    cd = str(geopdcoord)
    j = cd.replace("POINT (","") #remove palavra POINT (
    i = j.replace(")","").split(" ") # remove ultimo parenteses e splita o que sobrou resultando num array com as 2 coord
    ponto = Point(float(i[0]),float(i[1]))
    return ponto

In [52]:
roubox = []
rouboy = []
recupx = []
recupy = []

for roubo,recup in zip(dfRel['Geocode Novo'],dfRel['Geocode recup']):
    #print(recup)
    #break
    roubox.append(coordToPoint(roubo).x)
    rouboy.append(coordToPoint(roubo).y)
    recupx.append(coordToPoint(recup).x)
    recupy.append(coordToPoint(recup).y)

In [58]:
dfAnalise=dfRel.drop(columns=['Geocode Novo','Geocode recup'])
dfAnalise['x roubo'] = roubox
dfAnalise['y roubo'] = rouboy
dfAnalise['x recuperacao'] = recupx
dfAnalise['y recuperacao'] = recupy

In [60]:
dfAnalise.head(3)

(46606, 19)

In [61]:
with pd.ExcelWriter('Relacao_Roubo_Recup.xlsx') as writer: 
    dfAnalise.to_excel(writer, sheet_name='Base')